# Apply matcher M to A, B

In [ ]:
import pandas as pd
import py_entitymatching as em
A = em.read_csv_metadata('../data/TableA.csv', key = 'ID')
B = em.read_csv_metadata('../data/TableB.csv', key = 'ID')
block_f = em.get_features_for_blocking(A, B)
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()
r = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Title"]), dlm_dc0(rtuple["Title"]))', block_t, block_s)
em.add_feature(block_f, 'Title_Title_jac_dlm_dc0_dlm_dc0', r)
ob = em.OverlapBlocker()
C = ob.block_tables(A, B, 'Author', 'Author', 
                    l_output_attrs=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details'], 
                    r_output_attrs=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details'], 
                    overlap_size = 2)
D = ob.block_candset(C, 'Title', 'Title', overlap_size = 4)
match_f = em.get_features_for_matching(A, B)
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
f1 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Title"]), dlm_dc0(rtuple["Title"]))', match_t, match_s)
f2 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Author"]), dlm_dc0(rtuple["Author"]))', match_t, match_s)
f3 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Publication"]), dlm_dc0(rtuple["Publication"]))', match_t, match_s)
f4 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Series"]), dlm_dc0(rtuple["Series"]))', match_t, match_s)
em.add_feature(match_f, 'Title_Title_jac_dlm_dc0_dlm_dc0', f1)
em.add_feature(match_f, 'Author_Author_jac_dlm_dc0_dlm_dc0', f2)
em.add_feature(match_f, 'Publication_Publication_jac_dlm_dc0_dlm_dc0', f3)
em.add_feature(match_f, 'Series_Series_jac_dlm_dc0_dlm_dc0', f4)
# Add blackbox feature

import re
# for Roman numerals matching
def Title_Title_blackbox_1(x, y):
    
    # get name attribute
    x_title = x['Title']
    y_title = y['Title']
    regex_roman = '\s+[MDCLXVI]+\s+'
    x_match = None
    y_match = None
    if re.search(regex_roman, x_title):
        x_match = re.search(regex_roman, x_title).group(0)
    if re.search(regex_roman, y_title):
        y_match = re.search(regex_roman, y_title).group(0)

    if x_match is None or y_match is None:
        return False
    else:
        return x_match == y_match

em.add_blackbox_feature(match_f, 'blackbox_1', Title_Title_blackbox_1)
match_f = match_f[(match_f['left_attribute'] != 'ID') & (match_f['left_attribute'] != 'ISBN')]
H = em.extract_feature_vecs(D, feature_table=match_f)
pred_table = rf.predict(table= H, 
                        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID'], 
                        target_attr='predicted_labels', 
                        return_probs=True, 
                        probs_attr='proba', 
                        append=True)

# Merge A, B
We can obtain a list of ltable_ID and rtable_ID that are predicted to be a match. Based on the list, we first append the remaining non-matching cases to table E. For those appeared in the list, we retain attributes with longer string length, i.e. embedded with more information.

In [ ]:
#AB_list=[['ltable_ID','rtable_ID'] for x in pred_table if predicted_labels==1]
#AB_dict =  {item[0]: item[1] for item in AB_list}
#BA_dict =  {item[1]: item[0] for item in AB_list}

attr=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details']
with open("table_E.csv", 'wb') as csvOut:
    writer = csv.writer(csvOut,delimiter = ',')
    for index, r in A.iterrows():
        row=[]
        if r['ID'] in AB_dict.keys():
            for j in attr:
                if len(r[j])<len(B.loc[B['ID'] == AB_dict[r['ID']]][j])
                    row.append(B.loc[B['ID'] == AB_dict[r['ID']]][j])
                else:
                    row.append(r[j])
            else:
                for j in attr:
                    row.append(r[j])
        writer.writerow(row)
        
    for index, r in B.iterrows():
        row=[]
        if r['ID'] not in BA_dict.keys():
            for j in attr:
                row.append(r[j])
        writer.writerow(row)